In [1]:
import torch
import torch.nn as nn
from funcs import *
from Models import modelpool
from Preprocess import datapool

bs = 128
l = 8
t = l * 2
tau = 6
device ="cuda" if torch.cuda.is_available() else "cpu"

list_data = ["cifar10", "cifar100"]
list_models = ["vgg16", "resnet20"]
list_neuron_models = ["ann", "if", "gn", "pgn"]
acc_results: dict[str, float] = { }
loss_results: dict[str, float] = { }

for data_name in list_data:
	# Initialize data
	train, test = datapool(data_name, bs)
	for model_name in list_models:
		print(f'eval model: {model_name}, dataset: {data_name}')
		for neuron_model in list_neuron_models:
			key = f'[{data_name}_{model_name}_{neuron_model}'

			# Initialize model
			model = modelpool(model_name, data_name)
			model = replace_maxpool2d_with_avgpool2d(model)
			model = replace_relu_with_qcfs(model, L=l)
			model.load_state_dict(torch.load(f'./saved_models/{model_name}-{data_name}.pth'))
			criterion = nn.CrossEntropyLoss()
			
			if neuron_model == "ann":
				model.to(device)
				acc, loss = eval_ann(test, model, criterion, device)
			else:
				model = replace_qcfs_with_sn(model, members=tau, sn_type=neuron_model)
				model.to(device)
				acc, loss = eval_snn(test, model, criterion, device, t)
				
			acc_results[key] = acc
			loss_results[key] = loss
			print(f'{neuron_model} acc: {acc}')

2024-07-23 12:17:47.065808: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-23 12:17:47.075666: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 12:17:47.089664: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 12:17:47.089695: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 12:17:47.099165: I tensorflow/core/platform/cpu_feature_gua

Files already downloaded and verified
Files already downloaded and verified
eval model: vgg16, dataset: cifar10


100%|██████████| 79/79 [00:02<00:00, 34.54it/s]


ann acc: 0.9404


100%|██████████| 79/79 [00:21<00:00,  3.65it/s]


...

pgn acc: [0.9388 0.9398 0.9397 0.9406 0.9403 0.9406 0.9405 0.9405 0.9407 0.9405
 0.9403 0.9402 0.9404 0.9404 0.9407 0.9404]
eval model: resnet20, dataset: cifar10


100%|██████████| 79/79 [00:01<00:00, 64.30it/s]


ann acc: 0.92


 92%|█████████▏| 73/79 [00:08<00:00,  8.56it/s]


KeyboardInterrupt: 

In [ ]:
for results in [acc_results, loss_results]:
    for key in results.keys():
    	print(f'{key}: {results[key]}')

In [ ]:
import matplotlib.pyplot as plt
for data_name in list_data:
	for model_name in list_models:
		plt.figure()
		plt.title(data_name + ", " + model_name)
		for i, neuron_model in enumerate(list_neuron_models):
			key = f'[{data_name}_{model_name}_{neuron_model}'
			acc = acc_results[key]
			print(neuron_model, type(acc))
			if acc is np.float64:
				plt.hlines(y=acc, xmin=0, xmax=t, label=neuron_model, color=f"C{i}")
			else:
				plt.plot(acc, label=neuron_model, color=f"C{i}")
		plt.legend()